<a href="https://colab.research.google.com/github/Cypher-A/ChatBot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install transformers
!pip install nltk
!pip install spacy
!pip install tensorflow
!pip install torch

In [ ]:
import pandas as pb
import numpy as np
import tensorflow as tf
import torch
from transformers import BertTokenizer, TFBertModel, GPT2Tokenizer, FlaxGPT2LMHeadModel
import nltk
import spacy

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Download the dataset
!wget https://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

# Unzip the dataset
!unzip cornell_movie_dialogs_corpus.zip -d cornell_data


--2024-10-06 16:15:14--  https://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.53
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip’

cornell_movie_dialo 100%[===================>]   9.46M  20.9MB/s    in 0.5s    

2024-10-06 16:15:15 (20.9 MB/s) - ‘cornell_movie_dialogs_corpus.zip’ saved [9916637/9916637]

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell_data/cornell movie-dialogs corpus/
  inflating: cornell_data/cornell movie-dialogs corpus/.DS_Store  
   creating: cornell_data/__MACOSX/
   creating: cornell_data/__MACOSX/cornell movie-dialogs corpus/
  inflating: cornell_data/__MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell_data/cornell movie-dialogs corpus/chameleons.pdf  
  inflating: corn

In [ ]:
import os

# Path to the dataset
data_path = 'cornell_data/cornell movie-dialogs corpus/'

# List files
for file in os.listdir(data_path):
    print(file)


.DS_Store
movie_titles_metadata.txt
movie_characters_metadata.txt
README.txt
movie_lines.txt
movie_conversations.txt
chameleons.pdf
raw_script_urls.txt


In [ ]:
import pandas as pd

# Load lines and conversations
def load_lines(file_path):
    lines = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        for line in file:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 5:
                lines.append({'line_id': parts[0], 'character': parts[3], 'text': parts[4]})
    return pd.DataFrame(lines)

def load_conversations(file_path):
    conversations = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        for line in file:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 4:
                conv = eval(parts[3])  # Convert string list to actual list
                conversations.append(conv)
    return conversations

# Load data
lines_df = load_lines(data_path + 'movie_lines.txt')
conversations = load_conversations(data_path + 'movie_conversations.txt')

print(f"Total lines: {len(lines_df)}")
print(f"Total conversations: {len(conversations)}")


Total lines: 304446
Total conversations: 83097


In [ ]:
# Create a mapping from line ID to text
line_dict = dict(zip(lines_df['line_id'], lines_df['text']))

# Extract Q-A pairs
qa_pairs = []
for conv in conversations:
    for i in range(len(conv)-1):
        if conv[i] in line_dict and conv[i+1] in line_dict: # Check if both line IDs exist in line_dict
            qa_pairs.append([line_dict[conv[i]], line_dict[conv[i+1]]])

# Create DataFrame
qa_df = pd.DataFrame(qa_pairs, columns=['Question', 'Answer'])
print(f"Total Q-A pairs: {len(qa_df)}")
qa_df.head()

Total Q-A pairs: 221282


,Question,Answer
0,Can we make this quick? Roxanne Korrine and A...,"Well, I thought we'd start with pronunciation,..."
1,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....
2,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...
3,You're asking me out. That's so cute. What's ...,Forget it.
4,"No, no, it's my fault -- we didn't have a prop...",Cameron.


In [ ]:
import re

def clean_text(text):
    text = text.lower()
    # Remove unnecessary characters
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Apply cleaning
qa_df['Question'] = qa_df['Question'].apply(clean_text)
qa_df['Answer'] = qa_df['Answer'].apply(clean_text)
qa_df.head()


,Question,Answer
0,can we make this quick roxanne korrine and and...,well i thought we would start with pronunciati...
1,well i thought we would start with pronunciati...,not the hacking and gagging and spitting part ...
2,not the hacking and gagging and spitting part ...,okay then how bout we try out some french cuis...
3,you are asking me out that is so cute what is ...,forget it
4,no no its my fault we did not have a proper in...,cameron


In [ ]:
# Install transformers library
!pip install transformers

# Import necessary modules
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


In [ ]:
# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Function to generate a response
def generate_response(user_input):
    # Encode the input and add end of string token
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append the new user input tokens to the chat history
    bot_input_ids = new_user_input_ids

    # Generate a response
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Decode the response
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

# Test the chatbot
user_input = "Hello, how are you?"
response = generate_response(user_input)
print(f"User: {user_input}")
print(f"Bot: {response}")


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


User: Hello, how are you?
Bot: I'm good, how are you?


In [ ]:
# Function to generate a response
def generate_response(user_input):
    # Encode the input and add end of string token
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append the new user input tokens to the chat history
    bot_input_ids = new_user_input_ids

    # Generate a response
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Decode the response
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

# Test the chatbot
user_input = "Hello, how are you?"
response = generate_response(user_input)
print(f"User: {user_input}")
print(f"Bot: {response}")


User: Hello, how are you?
Bot: I'm good, how are you?


In [ ]:
print("Start chatting with the bot (type 'exit' to stop)!")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break
    response = generate_response(user_input)
    print(f"Chatbot: {response}")


Start chatting with the bot (type 'exit' to stop)!
You: hello
Chatbot: Hello! :D
You: what are you doing
Chatbot: I'm not doing anything.
You: exit
Chatbot: Goodbye!


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Example evaluation
reference = "I'm doing great, thank you!"
candidate = generate_response("How are you?")
reference = [reference.split()]
candidate = candidate.split()
score = sentence_bleu(reference, candidate)
print(f"BLEU score: {score}")


BLEU score: 1.2183324802375697e-231


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00


In [ ]:
# Download the script
%%writefile app.py

import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model and tokenizer
@st.cache(allow_output_mutation=True)
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
    return tokenizer, model

tokenizer, model = load_model()

st.title("Advanced Conversational AI Chatbot 🤖")

# Initialize session state
if 'history' not in st.session_state:
    st.session_state.history = []

# User input
user_input = st.text_input("You:", "")

if user_input:
    # Encode input
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Generate response
    chat_history_ids = model.generate(new_user_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(chat_history_ids[:, new_user_input_ids.shape[-1]:][0], skip_special_tokens=True)

    # Update history
    st.session_state.history.append(("You", user_input))
    st.session_state.history.append(("Bot", response))

# Display chat history
for speaker, text in st.session_state.history:
    if speaker == "You":
        st.markdown(f"**{speaker}:** {text}")
    else:
        st.markdown(f"**{speaker}:** {text}")


Writing app.py


In [ ]:
from google.colab import files
files.download('app.py')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Clone your repository to Colab
!git clone https://github.com/Cypher-A/ChatBot.git


Cloning into 'ChatBot'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
Receiving objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)


In [ ]:
# Create a new Python file in the cloned repo
with open('/content/ChatBot/app.py', 'w') as f:
    f.write('print("Hello, Streamlit!")')


In [ ]:
# Set up your GitHub credentials (replace with your email and username)
!git config --global user.email "podar01arya@gmail.com"
!git config --global user.name "Cypher-A"


In [ ]:
# Navigate to your cloned repository
%cd /content/ChatBot/

# Stage all changes
!git add .

# Commit the changes with a message
!git commit -m "Add app.py for chatbot"

# Push the changes to GitHub
!git push


/content/ChatBot
[main 7867943] Add app.py for chatbot
 1 file changed, 1 insertion(+)
 create mode 100644 app.py
fatal: could not read Username for 'https://github.com': No such device or address
